In [2]:
import os
from collections import defaultdict
import numpy as np
import pandas as pd
import networkx as nx
import osmnx as ox

import build.graph_utils.graph_utils as graph_utils

/var/folders/3x/vv95p66j6658cvlngdhhfd1r0000gq/T/ipykernel_42054/2192525410.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
graph_utils.__dict__

{'__name__': 'build.graph_utils.graph_utils',
 '__doc__': 'Graph utils',
 '__package__': 'build.graph_utils',
 '__loader__': <_frozen_importlib_external.ExtensionFileLoader at 0x15d608390>,
 '__spec__': ModuleSpec(name='build.graph_utils.graph_utils', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x15d608390>, origin='/Users/a.ashabokov/Documents/github/graph_processing_tools/graph_isohrones/build/graph_utils/graph_utils.so'),
 'single_source_dijkstra': <function build.graph_utils.graph_utils.PyCapsule.single_source_dijkstra>,
 'multi_source_dijkstra': <function build.graph_utils.graph_utils.PyCapsule.multi_source_dijkstra>,
 '__file__': '/Users/a.ashabokov/Documents/github/graph_processing_tools/graph_isohrones/build/graph_utils/graph_utils.so'}

In [4]:
graph = ox.load_graphml("test_graph_data/tiva_region.graphml")
nodes_df, edges_df = ox.graph_to_gdfs(graph)

In [5]:
nodes_df.head()

,y,x,street_count,highway,geometry
osmid,,,,,
2124710929,50.614446,97.386298,1,NaN,POINT (97.38630 50.61445)
2511734347,50.588890,97.481463,3,NaN,POINT (97.48146 50.58889)
2511734360,50.590414,97.533801,3,NaN,POINT (97.53380 50.59041)
2511734445,50.590637,97.529634,4,NaN,POINT (97.52963 50.59064)
2511734504,50.590746,97.535976,3,NaN,POINT (97.53598 50.59075)


In [6]:
edges_df.head()

osmid  \
u          v           key                                                 
2124710929 2511738000  0    [202504192, 202504193, 202504194, 243816002]   
2511734347 10741401701 0               [243816009, 243815980, 243815988]   
           10741401841 0                                       243815980   
           10741401700 0                                      1155003958   
2511734360 2511734820  0                                       243815985   

                           bridge          highway  oneway       reversed  \
u          v           key                                                  
2124710929 2511738000  0      yes  [path, footway]   False  [False, True]   
2511734347 10741401701 0      yes            track   False          False   
           10741401841 0      NaN            track   False           True   
           10741401700 0      NaN            track   False          False   
2511734360 2511734820  0      NaN      residential   False          False   

                              length  \
u          v           key             
2124710929 2511738000  0    1422.554   
2511734347 10741401701 0     933.430   
           10741401841 0     300.802   
           10741401700 0    1406.467   
2511734360 2511734820  0     379.231   

                                                                     geometry  \
u          v           key                                                      
2124710929 2511738000  0    LINESTRING (97.38630 50.61445, 97.39014 50.614...   
2511734347 10741401701 0    LINESTRING (97.48146 50.58889, 97.48217 50.589...   
           10741401841 0    LINESTRING (97.48146 50.58889, 97.48067 50.588...   
           10741401700 0    LINESTRING (97.48146 50.58889, 97.47995 50.588...   
2511734360 2511734820  0    LINESTRING (97.53380 50.59041, 97.53321 50.590...   

                                           name maxspeed  ref lanes junction  \
u          v           key                                                     
2124710929 2511738000  0                    NaN      NaN  NaN   NaN      NaN   
2511734347 10741401701 0                    NaN      NaN  NaN   NaN      NaN   
           10741401841 0                    NaN      NaN  NaN   NaN      NaN   
           10741401700 0                    NaN      NaN  NaN   NaN      NaN   
2511734360 2511734820  0    Комсомольская улица      NaN  NaN   NaN      NaN   

                           service width tunnel access  
u          v           key                              
2124710929 2511738000  0       NaN   NaN    NaN    NaN  
2511734347 10741401701 0       NaN   NaN    NaN    NaN  
           10741401841 0       NaN   NaN    NaN    NaN  
           10741401700 0       NaN   NaN    NaN    NaN  
2511734360 2511734820  0       NaN   NaN    NaN    NaN

In [3]:
graph = ox.load_graphml("test_graph_data/tiva_region.graphml")
nodes_df, edges_df = ox.graph_to_gdfs(graph)
edges_df.reset_index(drop=False, inplace=True)
edges_df = edges_df[['u', 'v', 'length']]
edges_df = edges_df.sort_values("length").drop_duplicates(keep="first")
edges_df['u'] = edges_df['u'].astype(str)
edges_df['v'] = edges_df['v'].astype(str)
edges_df['length'] = edges_df['length'].astype(float)
edges_df.dropna(inplace=True)
# edges_df = edges_df.groupby(['u', 'v']).agg('first')
edges_df

,u,v,length
20424,8395624086,6308368309,0.056
16266,6308368309,8395624086,0.056
20992,8452168701,4113519640,0.178
10333,4113519640,8452168701,0.178
6227,409084387,8343879824,0.331
...,...,...,...
3196,1328782656,1845588962,62621.307
33510,3384073699,3383912716,84339.932
33464,3383912716,3384073699,84339.932
250,3225000321,3090156324,127715.862


In [4]:
weights = defaultdict(dict)

for _, row in edges_df.iterrows():
    weights[row['u']][row['v']] = row['length']

weights = dict(weights)

In [5]:
start_nodes_lst = nodes_df.reset_index(drop=False)['osmid'].astype(str).to_list()
len(start_nodes_lst)

16720

In [7]:
np.random.shuffle(start_nodes_lst)

In [17]:
from tqdm import tqdm

pbar = tqdm(start_nodes_lst[:1000])
res_lst = []

for start in pbar:
    pbar.set_postfix_str(start)
    res = graph_utils.single_source_dijkstra(weights, start, 10_000)
    res_lst.append(res)

  0%|          | 0/1000 [00:00<?, ?it/s]

 82%|████████▏ | 820/1000 [11:26<02:30,  1.19it/s, 6343333679] 


KeyboardInterrupt: 

In [10]:
%%time
res = graph_utils.multi_source_dijkstra(weights, start_nodes_lst, 5_000, 8)

CPU times: user 1h 39min 36s, sys: 35.4 s, total: 1h 40min 12s
Wall time: 14min 33s


In [8]:
from tqdm import tqdm

pbar = tqdm(start_nodes_lst[:1000])
res_lst = []

for start in pbar:
    pbar.set_postfix_str(start)
    res = graph_utils.single_source_dijkstra(weights, start, 10_000)
    res_lst.append(res)

  2%|▏         | 22/1000 [00:23<17:13,  1.06s/it, 1463982761] 


KeyboardInterrupt: 